In [1]:
from loguru import logger
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
root_url = "https://www.fifaindex.com"
page_url = "/teams/fifa21_486/"
page_number = 1
data = []


while True:
    
    page = requests.get(f"{root_url}{page_url}?page={page_number}") # Getting page HTML through request
    soup = BeautifulSoup(page.content, 'html.parser') # Parsing content using beautifulsoup 
    table = soup.find("table")
    rows = table.find_all("tr")
    for row in rows:
        try:
            team_data = {}

            team_data["Name"] = row.find(attrs={"data-title": "Name"}).get_text()
            team_data["League"] = row.find(attrs={"data-title": "League"}).get_text()
            team_data["Defense"] = int(row.find(attrs={"data-title": "DEF"}).get_text())
            team_data["Midfield"] = int(row.find(attrs={"data-title": "MID"}).get_text())
            team_data["Attack"] = int(row.find(attrs={"data-title": "ATT"}).get_text())
            team_data["Overall"] = int(row.find(attrs={"data-title": "OVR"}).get_text())
            team_data["Logo URL"] = row.find("img").get("src", "")

            team_rating = row.find(attrs={"data-title": "Team Rating"})
            full_stars = len(team_rating.select("i.fas.fa-star"))
            half_stars = len(team_rating.select("i.fas.fa-star-half-alt"))
            numeric_rating = full_stars + half_stars / 2

            team_data["Team_Rating"] = numeric_rating
        except AttributeError:
            continue
        else:
            data.append(team_data)

    next_page_btn = soup.find("a", text="Next Page")
    if not next_page_btn:
        # Reached last page
        break
    page_number += 1
    logger.info(f"Moving to next page - {page_number}")
            

2021-12-26 01:32:29.654 | INFO     | __main__:<module>:41 - Moving to next page - 2
2021-12-26 01:32:29.790 | INFO     | __main__:<module>:41 - Moving to next page - 3
2021-12-26 01:32:29.923 | INFO     | __main__:<module>:41 - Moving to next page - 4
2021-12-26 01:32:30.061 | INFO     | __main__:<module>:41 - Moving to next page - 5
2021-12-26 01:32:30.249 | INFO     | __main__:<module>:41 - Moving to next page - 6
2021-12-26 01:32:30.381 | INFO     | __main__:<module>:41 - Moving to next page - 7
2021-12-26 01:32:30.514 | INFO     | __main__:<module>:41 - Moving to next page - 8
2021-12-26 01:32:30.646 | INFO     | __main__:<module>:41 - Moving to next page - 9
2021-12-26 01:32:30.783 | INFO     | __main__:<module>:41 - Moving to next page - 10
2021-12-26 01:32:30.914 | INFO     | __main__:<module>:41 - Moving to next page - 11
2021-12-26 01:32:31.055 | INFO     | __main__:<module>:41 - Moving to next page - 12
2021-12-26 01:32:31.185 | INFO     | __main__:<module>:41 - Moving to nex

In [8]:
team_df = pd.DataFrame.from_records(data)
team_df

,Name,League,Defense,Midfield,Attack,Overall,Logo URL,Team_Rating
0,Liverpool,Premier League,80,83,86,85,https://fifastatic.fifaindex.com/FIFA21/images...,5.0
1,Manchester City,Premier League,83,86,85,85,https://fifastatic.fifaindex.com/FIFA21/images...,5.0
2,Real Madrid,LaLiga Santander,84,86,84,85,https://fifastatic.fifaindex.com/FIFA21/images...,5.0
3,FC Bayern München,Bundesliga,82,85,92,84,https://fifastatic.fifaindex.com/FIFA21/images...,5.0
4,Paris Saint-Germain,Ligue 1 Uber Eats,82,82,87,84,https://fifastatic.fifaindex.com/FIFA21/images...,5.0
...,...,...,...,...,...,...,...,...
678,Derry City,SSE Airtricity Lge,58,57,59,57,https://fifastatic.fifaindex.com/FIFA21/images...,0.5
679,Waterford FC,SSE Airtricity Lge,57,57,57,57,https://fifastatic.fifaindex.com/FIFA21/images...,0.5
680,Shelbourne FC,SSE Airtricity Lge,56,56,57,56,https://fifastatic.fifaindex.com/FIFA21/images...,0.5
681,Cork City,SSE Airtricity Lge,53,55,54,54,https://fifastatic.fifaindex.com/FIFA21/images...,0.5


In [11]:
team_df.to_csv("./team_data.csv")